# MNIST Neural Network with NumPy (Optimized)

Simple implementation of a neural network using NumPy for MNIST digit classification. This version achieves ~96% validation accuracy through optimizations including batch gradient descent and efficient matrix operations. Uses Jacobian Vector products for a very big speed boost, should be pretty much the best we can do without GPU.

## Architecture
- Input (784) → Hidden (256) → Output (10)
- ReLU + Softmax activations
- Cross-entropy loss
- Batch gradient descent with size 128

In [1]:
import numpy as np
import csv
import os


PROJECT_ROOT = "/home/nlin/workspace/code/projects/autograd_cpp"

In [2]:
def load_mnist_from_file(csvfile):
    reader = csv.reader(csvfile)

    data = []
    labels = []
    next(reader)
    for row in reader:
        row = [float(i) for i in row]
        data.append([i * 1.0 / 255 for i in row[1:]])
        labels.append([1 if row[0] == i else 0 for i in range(10)])

    n = len(data)
    xs = np.array(data)
    ys = np.array(labels)
    csvfile.close()

    return xs, ys, n


csvfile = open(os.path.join(PROJECT_ROOT, "data/mnist_train.csv"))
xs, ys, n = load_mnist_from_file(csvfile)

In [3]:
def softmax(X):
    max_X = np.max(X, axis=1, keepdims=True)
    eX = np.exp(X - max_X)
    sum_eX = np.sum(eX, axis=1, keepdims=True)
    return eX / sum_eX

In [4]:
W1 = np.random.rand(784, 256) - 0.5
W2 = np.random.rand(256, 10) - 0.5

In [5]:
tcsvfile = open(os.path.join(PROJECT_ROOT, "data/mnist_test_short.csv"))
txs, tys, tn = load_mnist_from_file(tcsvfile)

In [ ]:
BATCH_SIZE = 128
NUM_ITER = 100000
learning_rate = 1e-3

for it in range(NUM_ITER):
    batch_indices = np.random.choice(n, BATCH_SIZE, replace=False)
    batch_xs = xs[batch_indices]
    batch_ys = ys[batch_indices]

    A1 = np.matmul(batch_xs, W1)
    Z1 = np.where(A1 > 0, A1, 0)
    A2 = np.matmul(Z1, W2)
    Z2 = softmax(A2)

    loss = -np.sum(np.log(Z2 + 1e-10) * batch_ys)

    dLda2 = Z2 - batch_ys
    dLdw2 = np.matmul(Z1.T, dLda2)
    dLdz1 = np.matmul(dLda2, W2.T)
    dLda1 = np.multiply(dLdz1, np.where(A1 > 0, 1, 0))
    dLdw1 = np.matmul(batch_xs.T, dLda1)
    learning_rate = 1e-3

    W1 -= learning_rate * dLdw1
    W2 -= learning_rate * dLdw2

    y_pred = np.argmax(Z2, axis=1)
    y_true = np.argmax(batch_ys, axis=1)
    if it % 1000 == 0:
        print(
            f"Iteration {it}; Loss: {float(loss)}, Accuracy: {np.sum(y_pred == y_true) / BATCH_SIZE}"
        )

    if it % 10000 == 0:
        t_A1 = np.matmul(txs, W1)
        t_Z1 = np.where(t_A1 > 0, t_A1, 0)
        t_A2 = np.matmul(t_Z1, W2)
        t_Z2 = softmax(t_A2)

        t_y_pred = np.argmax(t_Z2, axis=1)
        t_y_true = np.argmax(tys, axis=1)
        print("Validation Accuracy: ", np.sum(t_y_pred == t_y_true) / tn)